# Intro To Image Classification using Remote Sensing Data   

This notebook will take you through an image classification example using models and data from the torch library. The written code covers the main components of a machine learning experiment (data preparation, model definition, evaluation scripts, etc), with tasks/questions embedded throughout to aid the user in understanding the results.    

To use the GPU in Google Colab: Runtime > Change Runtime type , then select "Python 3" and "T4 GPU"

If you would like to learn more about the dataset, check out the paper: https://arxiv.org/pdf/1709.00029.pdf    
Tutorial adapted from https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html 



##### Import necessary libraries


In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm #for-loop progress bar

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import torch.nn.functional as F

import ssl 
ssl._create_default_https_context = ssl._create_unverified_context #to download torch dataset


##### Define variables
Here we need to define where our data will be saved, batch_size (i.e. how many samples we load at once), as well dataset sizes for the training and testing sets


In [ ]:
root_folder = '.' #use current directory
batch_size = 4
train_size = .8
test_size = .2

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

##### Download Dataset to Root Folder

In [ ]:
#define transformations that each sample will undergo as it's loaded
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = torchvision.datasets.EuroSAT(root = root_folder, download=True, transform = transform) #takes about ~1min to download

In [ ]:

#generate a training/testing data split
generator = torch.Generator().manual_seed(40) #to reproduce results
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size],generator=generator)


#create dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
                                         
print('There are ' + str(len(train_dataset)) + ' training images')
print('There are ' + str(len(test_dataset)) + ' test images')
print('There are ' + str(  len(dataset.classes) ) + ' classes')
class_to_idx = dict(zip(range(len(dataset.classes)),dataset.classes))

print(class_to_idx)


#### Inspect one batch of samples

In [ ]:

def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get some random training images
data_iter = iter(train_loader)
images, labels = next(data_iter)

imshow(torchvision.utils.make_grid(images))

# print labels
print(' '.join(f'{class_to_idx[int(labels[j])]:5s}' for j in range(batch_size)))



### QUESTION: What's the Height/Width of one image?
Hint: `images` is a torch.Tensor that has a `shape` attribute

In [ ]:
# ''' Insert code here. 


# '''


### QUESTION: How many samples do we have per class?
Hint: checkout `dataset.samples`. Converting it to a numpy array may be useful too using `np.array(dataset.samples)`

### Extra: what do you think happens if we have over-represented classes?

In [ ]:
# ''' Insert code here


# all_labels = ??
# '''
plt.hist(all_labels, bins=list(range(11)))

#### Define our Nueral Network Model
This is the fun part! Here, we define a simple convolutional nueral network that outputs the class probabilities for each image

In [ ]:

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(3, 6, 5)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.fc1 = torch.nn.Linear(2704, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10) # the 10 here represents how many classes we are predicting
        # the outputs will look like [prob_class_0, prob_class_2,...,prob_class_9]

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.to(device)


In [ ]:
# define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

#### Training the Model
Run 2 epochs (i.e. passes through the dataset) to train the model

#### Extra: Plot the loss per epoch
Adapt the training script to collect the running loss at the end of each epoch and use `plt.plot(x,y)` 

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times. For the simple model, ~1 epoch takes about 15 seconds. For resnet, 1 epoch takes about ~2.5min

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

#### Evaluate the trained model on the test set
Here, we iterate through the test set and collect the amount of correct predictions

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in tqdm(test_loader):
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

### TASK: Go back and run the training cell again for two more epochs. How much does your model's accuracy improve?

### EXTRA: replace your simple net with a pre-trained model from the torchvision library:
```
#load a neural network
resnet = models.resnet50(pretrained=True)
#pretrained resnet outputs 1000 classes. Here we change that to the 10 we have in our dataset
resnet.fc.out_features=10
resnet.to(device)
```
Train this new model. How does classifation accuracy compare with the simple model?


### TASK: Plot a Confusion Matrix to visualize how the model performed for each class

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

Hint: First, we need to collect lists of ALL the truth labels and predictions from the test dataset.    
Adapt the evaluation code block to collect truth labels and predictions for the entire test set    
Look at the 'predicted' and 'labels' variables. 'predicted' and 'labels are both type torch.Tensor, torch.Tensor.tolist() may be useful


In [ ]:
# ''' Insert code here

#  preds = ?? 
#  truth = ??

# '''

In [ ]:
cm = confusion_matrix(truth, preds, labels=list(range(10)))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=list(range(10)))
disp.plot()
plt.show()